Metacell run

In [ ]:
# Load the MetaCell data (host cell atlas)
load("E:/scTriH2/metacell/mc.scdr_TrH2_it4.Rda")

# Check what objects were loaded
ls()

# Assuming the main metacell object is called 'metacell_host', inspect it
str(metacell_host)

First, download the database from the github repository of xgrau:
- https://github.com/xgrau/placozoa-cell-type-evolution-code
- the directory is scatlas_results/data/scdb/
Then, load the metacell data as in read.me of the repo
- my data key is scdr_TrH2

In [ ]:
library(metacell)
setwd("/mnt/d/scTriH2/metacell")
source("xboc_functions.R")
# initialise metacell database using its relative path:
metacell::scdb_init("/mnt/d/scTriH2/metacell/mc_unzipped/", force_reinit = TRUE)

# read the mat object using its ID to refer to it; there ara analogous files for metacells, etc
mat_no_bact <- scdb_mat("scdr_TrH2_it2")

# sparse matrix available in the mat@mat slot, in this case it contains x genes and y cells
dim(mat_no_bact@mat)
    # 13948 genes x 15788 cells

# likewise, you may want to load a mc object containing cell-to-metacell assignments
mc = metacell::scdb_mc("scdr_TrH2_it4")
# the mc@mc slot is a vector with all cells and their associated metacell
length(mc@mc)
# 15704 cells associated with metacells

ctt = read.table("annotation_mc.TrH2.it4.reordered.tsv", header = TRUE)
ctt_col = scr_load_cell_type_table("annotation_mc.TrH2.it4.reordered.tsv", mc)
head(ctt)
nrow(ctt["metacell"])
# 217 metacells present
unique(ctt["cell_type"])
                              cell_type
1                              lipophil
37               trans_lipophil_1_fibre
38               trans_lipophil_1_gland
40           trans_lipophil_1_epithelia
41                            unknown_1
46                          unknown_2.1
47                          unknown_2.2
51                          unknown_2.3
52                          unknown_2.4
59                                fibre
91                    trans_fibre_gland
92                trans_fibre_epithelia
94                              gland_1
120                epithelia_gland_like
130                 epithelia_unknown_1
131                   epithelia_ventral
157 trans_epithelia_ventral_peptidergic
159      trans_epithelia_dorsal_ventral
164               epithelia_dorsal_like
178                    epithelia_dorsal
194             peptidergic_progenitors
195                   peptidergic_alpha
197                    peptidergic_beta
198                   peptidergic_gamma
199              peptidergic_gamma_like
200                   peptidergic_delta
201                 peptidergic_epsilon
202                    peptidergic_zeta
203                     peptidergic_eta
205                   peptidergic_theta
206                    peptidergic_iota
209               peptidergic_iota_like
211                   peptidergic_kappa
213                  peptidergic_lambda
214                      peptidergic_mu
215                 peptidergic_mu_like
216                      peptidergic_nu
217                 peptidergic_omicron



Secondly, the single cell data matrix (in this case in seurat object) is loaded. 
- The scRNA data are merged and processed (normalization , PCA)

In [ ]:
library(Seurat)
# load the symbiont sc matrix
BiocManager::install("SingleCellExperiment")
library(SingleCellExperiment)
install.packages("SeuratObject")
summary_object_1 <- readRDS("/mnt/e/scTriH2/SEURAT/extracted/1.cellRangerCount/Seurat_outputs/merged_seurat_object_1.rds")
sce = as.SingleCellExperiment(summary_object_1, assay="RNA")

# or
merged_seurat <- readRDS("/mnt/d/scTriH2/SEURAT/extracted/1.cellRangerCount/Seurat_outputs/merged_seurat_object_1.rds")
sce = as.SingleCellExperiment(merged_seurat, assay="RNA")

bact_mat = scm_import_sce_to_mat(sce)
library(Matrix)

# change barcode names like in names(mc@mc)
colnames(bact_mat@mat) <- paste0("H2_1_ACME_10x_10kc_", colnames(bact_mat@mat))

# add up every genes contained in a cell(barcode)
grellia <- colSums(as.matrix(bact_mat@mat))

# add missing barcodes to the grellia and set values to 0 before summing
missing_barcodes <- setdiff(names(mc@mc), names(grellia))
grellia[missing_barcodes] <- 0

# Count bacterial signal per metacell
grellia_total <- tapply(grellia[names(mc@mc)], mc@mc, sum)


## First, visualize how many grellia signal is contained in each metacells
# bacterial signal per metacell, point size = log(signals per cell)
pdf("bacterial_signal_per_mc.pdf", h=8, w=8, useDingbats=F)

# Scatter plot of grellia_total
plot(grellia_total, pch=20, cex=log(grellia_total + 1), col=as.character(ctt$color))

# Add text labels
text(grellia_total, labels=names(grellia_total), col=ifelse(pmax(grellia_total)> 6,"black",alpha("black",0)))

# Extract unique cell types with over 50 bacterial signal and corresponding colors
filtered_cells <- names(grellia_total)[grellia_total > 50]
filtered_ctt <- ctt[filtered_cells, , drop = FALSE]

unique_cell_types <- unique(filtered_ctt$cell_type)
unique_colors <- filtered_ctt$color[match(unique_cell_types, filtered_ctt$cell_type)]


# Add legend with unique cell types
legend("topright", legend=unique_cell_types, col=unique_colors, pch=20, border="black", cex=1)

dev.off()


# sc bacterial signal 2D projections, absolute counts
mc2d <- scdb_mc2d("scdr_TrH2_it4")
grellia_mc2d_abs <- scp_plot_gene_2d_metacell_bacteria(mc2d,mc,grellia_total,out_fn="Grellia_2d_abs.png",plot_mc=F,log=F)
grellia_mc2d_mc_abs <- scp_plot_gene_2d_metacell_bacteria(mc2d,mc,grellia_total,out_fn="Grellia_2d_mc_abs.p
ng",plot_mc=T,log=F)

# metacell visualization
mc2d_plot <- mcell_mc2d_plot("scdr_TrH2_it4", legend_pos ="bot
tomright", colors = ctt$color, fig_fn="mc2d_plot.png")


mat_no_bact_counts <- as.matrix(mat_no_bact@mat)
host_counts <- as.matrix(mc@mc_fp)
mc_counts <- as.matrix(mat_no_bact_counts) %*% t(as.matrix(mc_fp)) # genes x metacells matrix
dim(mat_no_bact_counts)
dim(host_counts)

# Check if the genes are the same and in the same order
all(rownames(mat_no_bact_counts) == rownames(host_counts))

# If not, align them
mat_no_bact_counts <- mat_no_bact_counts[rownames(host_counts), ]

# Assuming 'mc@mc' is a vector of metacell assignments for each cell
# and 'mat_no_bact_counts' is a gene-by-cell matrix

# Aggregate host counts by metacell
mc_counts <- t(rowsum(t(mat_no_bact_counts), group = mc@mc))

# Check dimensions
dim(mc_counts)  # Should be genes x metacells

mat_no_bact_counts <- as.matrix(mat_no_bact_counts)
storage.mode(mat_no_bact_counts) <- "numeric"

mc_fp <- as.matrix(mc_fp)
storage.mode(mc_fp) <- "numeric"


mc_counts <- as.matrix(mat_no_bact_counts) %*% t(as.matrix(mc_fp)) # genes x metacells matrix



mc_sizes=colSums(mc_counts)
grellia_norm=grellia_total*100/(grellia_total+mc_sizes)



In [ ]:
# or also 

After counting:
Generate figures for the metacell bacterial signal


In [ ]:
# metacell bacterial signal
pdf("bacterial_signal_per_mc.pdf",h=8,w=8,useDingbats=F)
plot(grellia_norm, pch=20, cex=4, col=as.character(ctt$color))
text(grellia_norm, labels=names(grellia_norm), col=ifelse(pmax(grellia_norm)>4,"black",alpha("black",0)))
dev.off()

# sc bacterial signal 2D projections
mc2d <- scdb_mc2d(mc2d_id="scdr_TrH2_it4")
grellia_mc2d <- scp_plot_gene_2d_metacell_bacteria(mc2d,mc,grellia_norm,out_fn="Grellia_2d.png",plot_mc=F,log=F)

# fraction of infected/bacterial-containing cells per animal
cell_animal=as.vector(mat@cell_metadata[names(mc@mc),"dataset"])
names(cell_animal)=names(mc@mc)
frac_grellia_cells=table(cell_animal[grellia_positive_cells])*100/table(cell_animal)

pdf("Frac_infected_cells_per_animal.pdf",h=10,w=6,useDingbats=F)
par(mfrow=c(2,1))
barplot(frac_grellia_cells,ylab="% infected cells",col="gray30",ylim=c(0,10),main="Grellia")
dev.off()
